<a href="https://colab.research.google.com/github/dankodanny/fromthescratch/blob/master/10models_JACS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install optuna
!pip install rdkit

from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score  # Add this import

import optuna

from tqdm import tqdm
from collections import deque

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
from rdkit.Chem import MACCSkeys
import math

import pandas as pd
import numpy as np

def get_smiles_encodings_for_dataset(file_path):
    df = pd.read_csv(file_path)
    smiles_list = np.asanyarray(df.SMILES)
    s1_energy_list = np.asarray(df['Redox'])
    smiles_alphabet = list(set(''.join(smiles_list)))
    smiles_alphabet.append(' ')  # for padding
    largest_smiles_len = len(max(smiles_list, key=len))
    # print('Finished translating SMILES to SELFIES.')
    return smiles_list, smiles_alphabet, largest_smiles_len, s1_energy_list

def smile_to_hot(smile, alphabet, largest_smile_len):
    """Go from a single smile string to a one-hot encoding.
    """
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    # pad with ' '
    smile += ' ' * (largest_smile_len - len(smile))
    # integer encode input smile
    integer_encoded = [char_to_int[c] for char in smile for c in char]
    # one hot-encode input smile
    onehot_encoded = deque()
    for value in integer_encoded:
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    onehot_result = deque()
    # 265*40 -> 10600으로 변환
    for row in onehot_encoded:
        onehot_result.extend(row)
    return integer_encoded, np.array(list(onehot_result))

def multiple_smile_to_hot(smiles_list, largest_molecule_len, alphabet):
    """Convert a list of smile strings to a one-hot encoding
    Returned shape (num_smiles x len_of_largest_smile x len_smile_encoding)
    """
    hot_list = []
    for s in tqdm(smiles_list, desc="S2H"):
        _, onehot_encoded = smile_to_hot(s, largest_molecule_len, alphabet)
        hot_list.append(onehot_encoded)
    # hot_list = numpy_reshape(hot_list)
    return np.array(hot_list)

In [2]:
'''
Smiles 2 hot input
'''

# Create synthetic data for testing

path = './train.csv'
smiles_list, alphabet, largest_smile_len, train_y = get_smiles_encodings_for_dataset(path)
hot_list = multiple_smile_to_hot(smiles_list, alphabet, largest_smile_len)      # hot_list는 np.array

# X, Y setting
X, y = hot_list, np.array(train_y)

FileNotFoundError: [Errno 2] No such file or directory: './train.csv'

In [9]:
'''
Finger print input
'''

ffpp = "pattern"

train = pd.read_csv("./train.csv")

train_fps = []#train fingerprints
train_y = [] #train y(label)

for index, row in train.iterrows() :
  try :
    mol = Chem.MolFromSmiles(row['SMILES'])
    if ffpp == 'maccs' :
        fp = MACCSkeys.GenMACCSKeys(mol)
    elif ffpp == 'morgan' :
        fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 4)
    elif ffpp == 'rdkit' :
        fp = Chem.RDKFingerprint(mol)
    elif ffpp == 'pattern' :
        fp = Chem.rdmolops.PatternFingerprint(mol)
    elif ffpp == 'layerd' :
        fp = Chem.rdmolops.LayeredFingerprint(mol)

    train_fps.append(fp)
    train_y.append(row['Redox'])
  except :
    pass

#fingerfrint object to ndarray
np_train_fps = []
for fp in train_fps:
  arr = np.zeros((0,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  np_train_fps.append(arr)

np_train_fps_array = np.array(np_train_fps)

X, y = np_train_fps_array , np.array(train_y)


In [ ]:
# Define models and their corresponding objective functions
models = {
    "LinearRegression": (LinearRegression, {"fit_intercept": None, "positive": None}),
    "Ridge": (Ridge, {"alpha": None}),
    "DecisionTreeRegressor": (DecisionTreeRegressor, {"criterion": None, "max_depth": None, "min_samples_leaf": None}),
    "SVR": (SVR, {"kernel": None, "degree": None}),
    "KNeighborsRegressor": (KNeighborsRegressor, {"n_neighbors": None, "weights": None}),
    "RandomForestRegressor": (RandomForestRegressor, {"n_estimators": None, "criterion": None, "max_depth": None, "min_samples_leaf": None}),
    "AdaBoostRegressor": (AdaBoostRegressor, {"n_estimators": None, "learning_rate": None, "loss": None}),
    "GradientBoostingRegressor": (GradientBoostingRegressor, {"n_estimators": None, "learning_rate": None, "loss": None}),
    "BaggingRegressor": (BaggingRegressor, {"n_estimators": None, "max_samples": None, "max_features": None}),
    "ExtraTreesRegressor": (ExtraTreesRegressor, {"criterion": None, "max_depth": None, "min_samples_leaf": None}),
    "MLPRegressor": (MLPRegressor, {"n_layers": None, "layers": None, "lr": None, "alpha": None}),
    "LGBMRegressor": (LGBMRegressor, {"num_leaves": None, "min_data_in_leaf": None, "max_depth": None, "learning_rate": None, "max_bin": None}),
}

def objective(trial, model_name):
    model_class, param_grid = models[model_name]

    model_params = {}
    if model_name == "MLPRegressor":
        n_layers = trial.suggest_int('n_layers', 1, 5)
        layers = [int(trial.suggest_loguniform('n_units_l{}'.format(i), 4, 200)) for i in range(n_layers)]
        lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
        alpha = trial.suggest_loguniform('alpha', 1e-6, 0.1)
        model_params["n_layers"] = n_layers
        model_params["layers"] = layers
        model_params["lr"] = lr
        model_params["alpha"] = alpha

    elif model_name == "LGBMRegressor":
        num_leaves = trial.suggest_int('num_leaves', 5, 100)
        min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 100)
        max_depth = trial.suggest_int('max_depth', -1, 10)
        lr = trial.suggest_loguniform('lr', 1e-5, 1.0)
        max_bin = trial.suggest_int('max_bin', 10, 500)
        model_params["num_leaves"] = num_leaves
        model_params["min_data_in_leaf"] = min_data_in_leaf
        model_params["max_depth"] = max_depth
        model_params["lr"] = lr
        model_params["max_bin"] = max_bin

    else:
        # Add your code block here for other models
        if model_name == "LinearRegression":
            fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
            positive = trial.suggest_categorical("positive", [True, False])
            model_params["fit_intercept"] = fit_intercept
            model_params["positive"] = positive

        elif model_name == "DecisionTreeRegressor":
            criterion = trial.suggest_categorical("criterion", ["squared_error", "friedman_mse", "absolute_error"])
            max_depth = trial.suggest_int("max_depth", 1, 10)
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
            model_params["criterion"] = criterion
            model_params["max_depth"] = max_depth
            model_params["min_samples_leaf"] = min_samples_leaf

        elif model_name == "SVR":
            kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly"])
            degree = trial.suggest_int("degree", 2, 5)
            model_params["kernel"] = kernel
            model_params["degree"] = degree

        elif model_name == "KNeighborsRegressor":
            n_neighbors = trial.suggest_int("n_neighbors", 1, 10)
            weights = trial.suggest_categorical("weights", ["uniform", "distance"])
            model_params["n_neighbors"] = n_neighbors
            model_params["weights"] = weights

        elif model_name == "RandomForestRegressor":
            n_estimators = trial.suggest_int("n_estimators", 10, 200)
            criterion = trial.suggest_categorical("criterion", ["squared_error", "friedman_mse", "absolute_error"])
            max_depth = trial.suggest_int("max_depth", 1, 10)
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
            model_params["n_estimators"] = n_estimators
            model_params["criterion"] = criterion
            model_params["max_depth"] = max_depth
            model_params["min_samples_leaf"] = min_samples_leaf

        elif model_name == "AdaBoostRegressor":
            n_estimators = trial.suggest_int("n_estimators", 10, 200)
            learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 1.0)
            loss = trial.suggest_categorical("loss", ["linear", "square", "exponential"])
            model_params["n_estimators"] = n_estimators
            model_params["learning_rate"] = learning_rate
            model_params["loss"] = loss

        elif model_name == "GradientBoostingRegressor":
            n_estimators = trial.suggest_int("n_estimators", 10, 200)
            learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 1.0)
            loss = trial.suggest_categorical("loss", ['squared_error', 'quantile', 'huber', 'absolute_error'])
            model_params["n_estimators"] = n_estimators
            model_params["learning_rate"] = learning_rate
            model_params["loss"] = loss

        elif model_name == "BaggingRegressor":
            n_estimators = trial.suggest_int("n_estimators", 10, 200)
            max_samples = trial.suggest_uniform("max_samples", 0.1, 1.0)
            max_features = trial.suggest_uniform("max_features", 0.1, 1.0)
            model_params["n_estimators"] = n_estimators
            model_params["max_samples"] = max_samples
            model_params["max_features"] = max_features

        elif model_name == "ExtraTreesRegressor":
            criterion = trial.suggest_categorical("criterion", [ 'friedman_mse', 'squared_error', 'absolute_error'])
            max_depth = trial.suggest_int("max_depth", 1, 10)
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
            model_params["criterion"] = criterion
            model_params["max_depth"] = max_depth
            model_params["min_samples_leaf"] = min_samples_leaf

        elif model_name == "Ridge":
            alpha = trial.suggest_loguniform('alpha', 1e-4, 10)
            model_params["alpha"] = alpha

        else:
            raise ValueError(f"Invalid model_name: {model_name}")

    # Create the model with optimized hyperparameters
    model = model_class(**model_params)

    # Perform cross-validation and return negative mean absolute error (MAE)
    mae_values = -cross_val_score(model, X, y, cv=4, scoring="neg_mean_absolute_error")
    return np.mean(mae_values)

# Create an Optuna study for each model
study_results = {}
for model_name in models.keys():
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, model_name), n_trials=10)
    study_results[model_name] = study

# Print the optimized hyperparameters for each model
for model_name, study in study_results.items():
    print(f"{model_name}에 대한 최적 하이퍼파라미터:")
    for key, value in study.best_params.items():
        print(f"{key}: {value}")
    print()


[I 2024-01-11 02:26:31,667] A new study created in memory with name: no-name-0ff55e33-bcb1-408e-923d-97eccf25a4a7
[I 2024-01-11 02:26:48,384] Trial 0 finished with value: 0.7976786938254727 and parameters: {'fit_intercept': True, 'positive': True}. Best is trial 0 with value: 0.7976786938254727.
[I 2024-01-11 02:27:02,456] Trial 1 finished with value: 0.785315528609059 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 1 with value: 0.785315528609059.
[I 2024-01-11 02:27:17,725] Trial 2 finished with value: 0.7976786938254727 and parameters: {'fit_intercept': True, 'positive': True}. Best is trial 1 with value: 0.785315528609059.
[I 2024-01-11 02:27:32,986] Trial 3 finished with value: 0.7976786938254727 and parameters: {'fit_intercept': True, 'positive': True}. Best is trial 1 with value: 0.785315528609059.
[I 2024-01-11 02:27:44,734] Trial 4 finished with value: 344454419.15836567 and parameters: {'fit_intercept': True, 'positive': False}. Best is trial 1 with 

In [7]:
import copy
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_regression(model, X_train, X_val, X_test, y_train, y_val, y_test):
    reg = copy.deepcopy(model)
    reg.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
    y_pred = reg.predict(X_test)
    rmse = mean_squared_error(y_pred, y_test)**0.5
    r2 = r2_score(y_pred, y_test)
    return rmse, r2

# Define models with optimized hyperparameters
models = {
    "Linear Regression": LinearRegression(fit_intercept=False, positive=True),
    "Decision Tree Regressor": DecisionTreeRegressor(criterion='friedman_mse', max_depth=6, min_samples_leaf=8),
    "SVR": SVR(kernel='rbf', degree=4),
    "K Neighbors Regressor": KNeighborsRegressor(n_neighbors=6, weights='distance'),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=115, criterion='friedman_mse', max_depth=9, min_samples_leaf=6),
    "AdaBoost Regressor": AdaBoostRegressor(n_estimators=101, learning_rate=0.3657, loss='exponential'),
    "Gradient Boosting Regressor": GradientBoostingRegressor(n_estimators=186, learning_rate=0.2717, loss='squared_error'),
    "Bagging Regressor": BaggingRegressor(n_estimators=165, max_samples=0.9579, max_features=0.8558),
    "Extra Trees Regressor": ExtraTreesRegressor(criterion='absolute_error', max_depth=6, min_samples_leaf=8),
    "MLP Regressor": MLPRegressor(
        hidden_layer_sizes=(120,38,102),
        #activation='tanh',
        #solver='lbfgs',
        alpha=8e-06,
        #learning_rate='invscaling',
        learning_rate_init=0.0007
    )
}

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Evaluate each model
for model_name, model in models.items():
    rmse, r2 = evaluate_regression(model, X_train, X_val, X_test, y_train, y_val, y_test)
    print(f"{model_name}: RMSE value for test set: {rmse}, R^2 value: {r2}")

    # Save the model
    joblib.dump(model, f"{model_name}_model.joblib")

Linear Regression: RMSE value for test set: 0.8062769641788758, R^2 value: 0.12332637739793917
Decision Tree Regressor: RMSE value for test set: 0.7570061552533178, R^2 value: -0.625027096396116
SVR: RMSE value for test set: 0.6238500514747718, R^2 value: -0.39130691978513354
K Neighbors Regressor: RMSE value for test set: 0.7796856206749837, R^2 value: -1.5963027906578544
Random Forest Regressor: RMSE value for test set: 0.6619357899963417, R^2 value: -0.6302610130631294
AdaBoost Regressor: RMSE value for test set: 0.7179579637293303, R^2 value: -2.4817610347160994
Gradient Boosting Regressor: RMSE value for test set: 0.5964413269058734, R^2 value: 0.33006371030944015
Bagging Regressor: RMSE value for test set: 0.6140184138038319, R^2 value: -0.20957966014894125
Extra Trees Regressor: RMSE value for test set: 0.7500779767671344, R^2 value: -0.7385898857660012
MLP Regressor: RMSE value for test set: 0.6183894886210476, R^2 value: 0.41999649489387814
